In [ ]:
import random

# Step 1: Generate reference DNA sequence
def generate_reference(length=1000, cg_ratio=0.1):
    seq = ''
    i = 0
    while i < length:
        if random.random() < cg_ratio and i < length - 1:
            seq += 'CG'
            i += 2
        else:
            seq += random.choice('AT')
            i += 1
    return seq

# Step 2: Simulate methylation map
def simulate_methylation(seq, methylation_rate=0.2):
    methylation_map = {}
    for i in range(len(seq) - 1):
        if seq[i:i+2] == 'CG':
            methylation_map[i] = random.random() < methylation_rate
    return methylation_map

# Step 3: Simulate bisulfite conversion
def bisulfite_convert(seq, methylation_map, efficiency=0.95):
    converted = list(seq)
    for i in range(len(seq)):
        if seq[i] == 'C':
            is_methylated = methylation_map.get(i, False)
            if not is_methylated and random.random() < efficiency:
                converted[i] = 'T'
    return ''.join(converted)

# Step 4: Simulate reads (list of [read_id, sequence])
def simulate_reads(converted_seq, read_length=100, coverage=10):
    reads = []
    for i in range(coverage):
        start = random.randint(0, len(converted_seq) - read_length)
        read_seq = converted_seq[start:start+read_length]
        reads.append((f"read_{start}", read_seq, start))
    return reads

# Step 5: Estimate conversion efficiency
def estimate_efficiency(reference, reads, methylation_map):
    total_unmethylated = 0
    converted = 0

    for read_id, read_seq, start in reads:
        for i, base in enumerate(read_seq):
            ref_pos = start + i
            if ref_pos >= len(reference): continue
            ref_base = reference[ref_pos]
            is_methylated = methylation_map.get(ref_pos, False)

            if ref_base == 'C' and not is_methylated:
                total_unmethylated += 1
                if base == 'T':
                    converted += 1

    if total_unmethylated == 0:
        return 0.0
    return converted / total_unmethylated

# Run simulation
reference = generate_reference()
methylation_map = simulate_methylation(reference, methylation_rate=0.2)
true_efficiency = 0.90
converted_seq = bisulfite_convert(reference, methylation_map, efficiency=true_efficiency)
reads = simulate_reads(converted_seq, coverage=30)

estimated_efficiency = estimate_efficiency(reference, reads, methylation_map)

# Show results
print(f"True efficiency: {true_efficiency*100:.1f}%")
print(f"Estimated efficiency: {estimated_efficiency*100:.1f}%")